In [1]:
import pandas as pd
import io

# 取得資料集
df = pd.read_csv("letter_train.csv")
print('資料的形式:{}'.format(df.shape))
print('遺漏的數量:{}'.format(df.isnull().sum().sum()))
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'letter_train.csv'

In [ ]:
df["class"].value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data_le=pd.DataFrame(df["class"])
df['class'] = labelencoder.fit_transform(data_le['class'])
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#印出每一個種類的數量
sns.countplot(x=df["class"], palette="Set3")
plt.title("class ",fontsize=10)
plt.show()

In [ ]:
#從dataframe中取出feature和label
x = df.drop(['class'], axis = 1)
print(x)
y = df.loc[:,'class'].values
print(y)

In [ ]:
#將feature標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)

In [ ]:
# 設定亂數產生器的起始點(包含Numpy與Tensorflow)
seed = 42#42
from numpy import random
random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

In [ ]:
# 載入使用的套件及子套件
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout #Dropout for solving overfitting

In [ ]:
# 將資料集分為訓練資料與測試資料
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.3, random_state=42) #test_size=0.2
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, stratify=y_train, test_size=0.3, random_state=42) #plus divide the training data into valid data.
print(x.shape)
print(X_train.shape)
print(X_test.shape)
print(y.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
pd.Series(y).value_counts(normalize=True)

In [ ]:
pd.Series(y_train).value_counts(normalize=True)


In [ ]:
from keras import regularizers
from keras.constraints import max_norm
from keras.layers import PReLU, ELU, BatchNormalization
model = Sequential()
model.add(Dense(units=256, input_dim=X_train.shape[1],
                kernel_initializer='uniform',
                activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units=128,
                kernel_initializer='uniform',
                activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(units=64,
                kernel_initializer='uniform',kernel_constraint=max_norm(5)))
model.add(ELU())
model.add(Dropout(0.125))
model.add(BatchNormalization())
model.add(Dense(units=26,
                kernel_initializer='uniform',
                activation='softmax',
                kernel_regularizer=regularizers.l2(0.05)))
model.summary()

In [ ]:
# 編譯模型
import tensorflow as tf

#sparse_categorical_crossentropy
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [ ]:
# 訓練模型
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)

train_history = model.fit(X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=200,
    batch_size=16,
    callbacks=[early_stopping,reduce_lr]
)


In [ ]:
plt.plot(train_history.history['loss'], label="Train")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()

In [ ]:
plt.plot(train_history.history['accuracy'], label="Train")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.show()

In [ ]:
scores = model.evaluate(x=X_test, y=y_test)

In [ ]:
pred = model.predict(X_test)

In [ ]:
y_pred=[]
for y in pred:
  y_pred.append(np.argmax(y))

In [ ]:
y_pred = np.array(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, labels=[0, 1, 2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]))

In [ ]:
import pandas as pd
import io

# 取得資料集
df_predict = pd.read_csv("letter_predict.csv")
print('資料的形式:{}'.format(df_predict.shape))
print('遺漏的數量:{}'.format(df_predict.isnull().sum().sum()))
df_predict.head()

In [ ]:
# 正規化屬性數值
X_predict=df_predict
scaler=StandardScaler()
X_predict=scaler.fit_transform(X_predict)
prediction = np.argmax(model.predict(X_predict), axis=1)


In [ ]:
#將結果寫入 csv 檔

with open("predict.csv", 'w') as f:
    f.write('id,Class\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i,chr(y+97).upper()))